In [3]:
# import public packages
import os
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib.patches import Rectangle
import scipy
import mne
import sys

from mne.time_frequency import tfr_morlet
from mne.baseline import rescale
from scipy.signal import spectrogram, hann, butter, filtfilt, hilbert
from scipy import signal, interpolate, stats
from scipy.interpolate import make_interp_spline, BSpline
from io import open
from importlib import reload

# import own functions
from utils import find_folders
import dat_preproc
import fix_annot_onsets
import mat2fif
import baseline_correction
import normalization

### Add Directories/ Load Files

In [4]:
reload(find_folders)
onedrive = find_folders.get_onedrive_path()
project_path = find_folders.get_onedrive_path("entrainment")
print(project_path)

/Users/barbaramathiopoulou/OneDrive - Charité - Universitätsmedizin Berlin/ENTRAINMENT_PROJECT


In [ ]:
fname = 'sub035_ses-2022011308165394_run-BrainSense20220113084500.mat'

raw = mne.io.read_raw_fieldtrip(
    os.path.join(
        project_path,
        'data',
        'raw_data',
        'test',
        fname
    ),
    info = None
)

subID = 'Sub035'

In [ ]:
reload(dat_preproc)
%matplotlib qt
matplotlib.rc('font', size=10)
x = raw.get_data(reject_by_annotation = 'omit',picks=[0,1])
raw = raw
win_samp = 250
noverlap = 0.5
window = hann(win_samp, sym=False)
f, t, Sxx = dat_preproc.fft_rawviz(raw, x, win_samp, noverlap)

In [ ]:
fft_name = str(subID)+'_FFT'
print(fft_name)

In [ ]:
#Saving the figures & files
fft_fig = os.path.join(project_path, 'figures','FFts','test/')
fft_file = os.path.join(project_path, 'data','FFTs','test/')

plt.savefig(str(fft_fig)+str(fft_name),dpi = 300)

np.save(str(fft_file)+str(fft_name)+'.npy',Sxx)

In [ ]:
#Saving the raw data as a fif file
raw2 = raw.copy()
Draw = raw2.get_data()
info = raw.info
new_raw = mne.io.RawArray(Draw, info)

fif_name = os.path.join(project_path, 'data','Fifs','test/',str(subID)+'_FIF.fif')
new_raw.save(fif_name)


In [ ]:
''''
test_raw = mne.io.read_raw_fif(os.path.join(
    project_path,
        'data',
        'Fifs',
        'test',
        'Sub005_FIF.fif'
    )
)
'''

In [ ]:
#Zscore Normalization of raw data & plot of new figure
reload(baseline_correction)
data = Sxx[1,:,]
t = t
baseline = (None, None)
raw = new_raw
stim_ch = 1
bs_data = baseline_correction.baseline_corr(data, t, baseline, raw = new_raw, stim_ch = 5)

In [ ]:
fft_fig = os.path.join(project_path, 'figures','FFts','test/')
fft_file = os.path.join(project_path, 'data','FFTs','test/')

plt.savefig(str(fft_fig)+str(fft_name)+'_zscored',dpi = 300)
np.save(str(fft_file)+str(fft_name)+'_zscored'+'.npy',bs_data)

In [ ]:
d = {'onset': [370, 542, 816],
    'duration': [30, 30, 10],
    'description': ['StimOff','StimOn','Rebound']
}

epoch_df = pd.DataFrame(data = d)
print(epoch_df)

In [ ]:
## Print Power Spectra with Raw Data
reload(dat_preproc)
filt_dat = raw.get_data()
time_onsets = epoch_df
window = 250
noverlap = 0.5
ylim2 = 0.2

fig = plt.figure(figsize = (12,5))

plt.subplot(1,2,1)
title1 = str(subID)+ '_LSTN'
ps1 = dat_preproc.epoch_ps(filt_dat, time_onsets, window, noverlap, 0, 0.2, title1)

plt.subplot(1,2,2)
title2 = str(subID)+ '_RSTN'
ps2 = dat_preproc.epoch_ps(filt_dat, time_onsets, window, noverlap, 1, 0.4, title2)

plt.show()

In [ ]:
## Printing the Power Spectra (RAW)
ps_all = np.concatenate((ps1,ps2), axis = 0)
ps_df = pd.DataFrame(np.transpose(ps_all),
    columns = ['StimOff_LSTN','StimOn_LSTN','Rebound_LSTN',
    'StimOff_RSTN','StimOn_RSTN','Rebound_RSTN'],
    )
print(ps_df)

In [ ]:
#Saving the figures & files
fft_fig = os.path.join(project_path, 'figures','Power_Spectra','test/')
fft_file = os.path.join(project_path, 'data','power_spectra','test/')

plt.savefig(str(fft_fig)+str(fft_name)+'_PS',dpi = 150)
ps_df.to_csv(str(fft_file)+str(fft_name)+'_PS.csv')

In [ ]:
## Print Power Spectra with Zscored Data
fig = plt.figure(figsize = (5,5))
ps_m1s0 = np.mean(bs_data[:,epoch_df.onset[0]:epoch_df.onset[0]+ epoch_df.duration[0]],1)
ps_m1s1 = np.mean(bs_data[:,epoch_df.onset[1]:epoch_df.onset[1]+ epoch_df.duration[1]],1)
ps_reb = np.mean(bs_data[:,epoch_df.onset[2]:epoch_df.onset[2]+ epoch_df.duration[2]],1)

plt.plot(np.arange(1,127),ps_m1s0,label = epoch_df.description[0])
plt.plot(np.arange(1,127),ps_m1s1,label = epoch_df.description[1])
plt.plot(np.arange(1,127),ps_reb,label = epoch_df.description[2])

plt.legend()

plt.xlim(60,90)
plt.xlabel('Frequency [Hz]')
plt.ylabel('LFP Power (z-scored)')
plt.show()

ps_allzscored = np.transpose(np.squeeze(np.array([[ps_m1s0], [ps_m1s1],[ps_reb]])))
ps_allzscored_df = pd.DataFrame(ps_allzscored,
    columns = ['StimOff','StimOn','Rebound'],
    )
print(ps_allzscored_df)

In [ ]:
plt.savefig(str(fft_fig)+str(fft_name)+'_zscoredPS',dpi = 150)
ps_allzscored_df.to_csv(str(fft_file)+str(fft_name)+'_zscoredPS.csv')

### Average all Power Spectra and Plot them

In [ ]:
all_psSpontan = pd.read_csv(os.path.join(
    project_path,
        'results',
        'AllSpontan_PsZscored.csv'
))


all_psSubharm = pd.read_csv(os.path.join(
    project_path,
        'results',
        'AllSubharm_PsZscored.csv'
))

In [ ]:
all_psSpontan = all_psSpontan.drop('Sub029', axis=1)

In [ ]:
all_ps_mean1 = np.mean(all_psSpontan,1)
all_ps_sem1 = stats.sem(all_psSpontan,1)

all_ps_mean2 = np.mean(all_psSubharm,1)
all_ps_sem2 = stats.sem(all_psSubharm,1)

In [ ]:
plt.plot(np.arange(1,127), all_ps_mean1, label = 'Spontaneous FTG')
plt.fill_between(np.arange(1,127), all_ps_mean1-all_ps_sem1, all_ps_mean1+all_ps_sem1, alpha = 0.2)

plt.plot(np.arange(1,127), all_ps_mean2, label = 'DBS-Induced FTG')
plt.fill_between(np.arange(1,127), all_ps_mean2-all_ps_sem2, all_ps_mean2+all_ps_sem2, alpha = 0.2)

plt.xlim(50, 100)
plt.ylim(-0.4, 2)
plt.legend()

plt.xlabel('Frequency [Hz]')
plt.ylabel('LFP Power [z-scored]')
plt.title('Averaged Power Spectra [N = 6]')

In [ ]:
plt.savefig(os.path.join(
    project_path,
    'results',
    'Avg_PS'
),
dpi = 200)

### Select Peaks and Plot Analytic Signal

In [5]:
#Load raw fif data

test_raw = mne.io.read_raw_fif(os.path.join(
    project_path,
        'data',
        'Fifs',
        'test',
        'Sub017_FIF.fif'
    )
)

subID = 'Sub017'
fft_name = str(subID) + '_'
print(fft_name)

Opening raw data file /Users/barbaramathiopoulou/OneDrive - Charité - Universitätsmedizin Berlin/ENTRAINMENT_PROJECT/data/Fifs/test/Sub017_FIF.fif...
Isotrak not found
    Range : 0 ... 94874 =      0.000 ...   379.496 secs
Ready.
Sub017_


<ipython-input-5-4e5cb0af6d14>:8: RuntimeWarning: This filename (/Users/barbaramathiopoulou/OneDrive - Charité - Universitätsmedizin Berlin/ENTRAINMENT_PROJECT/data/Fifs/test/Sub017_FIF.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  'Sub017_FIF.fif'


In [14]:
#Load Power Spectra 
%matplotlib qt
ps_zscored = pd.read_csv(os.path.join(
    project_path,
        'data',
        'power_spectra',
        'test',
        'Sub017_FFT_zscoredPS.csv'
))
plt.plot(ps_zscored['StimOff'], label = 'Med On - Stim Off')
plt.plot(ps_zscored['StimOn'], label  = 'Med On - Stim On')
plt.legend(loc='upper right')
plt.xlabel('Frequency [Hz]')
plt.ylabel('LFP Power [zscored]')
plt.xlim(50,90)
plt.ylim(-0.6, 1.3)


(-0.6, 1.3)

In [15]:
plt.savefig('Sub017_Exemplary_ZscoredPS',dpi = 150)

In [19]:
reload(dat_preproc)
x = test_raw.get_data() #getting only RSTN channel
x1 = x[1,:] 

dat_ngam = dat_preproc.low_highpass_filter(x1, 75, 79) 
dat_subh = dat_preproc.low_highpass_filter(x1, 61, 65) 

datall = [dat_ngam, dat_subh] 
labels = ['Peak 77Hz','Peak 63Hz']

In [7]:
def window_rms(a, window_size):
  a2 = np.power(a,2)
  window = np.ones(window_size)/float(window_size)
  return np.sqrt(np.convolve(a2, window, 'valid'))

In [9]:
sm_signal.shape

(94376,)

In [10]:
%matplotlib qt
sm_signal_np = np.empty(shape = (2, 94376))
sm_signal_np[:] = np.nan

fig, axes = plt.subplots(1, 1, figsize=(12, 5))
wintosmooth = 500

for idx, dat in enumerate(datall):
    hiltr = hilbert(dat)
    amplitude_envelope = np.abs(hiltr)
    zscore_sign = stats.zscore(np.squeeze(amplitude_envelope))

    sm_signal = window_rms(zscore_sign, wintosmooth)
    
    plt.plot(sm_signal, label = labels[idx])
    #plt.plot(np.arange(0,75000), amplitude_envelope, label = labels[idx]) 
    
    #axes[idx].axvline(26250, color = 'b', ls='--', lw=2, label = 'Stim On')
    #axes[idx].axvline(50250, color = 'g', ls='--', lw=2, label = 'Stim Off')
    plt.ylabel('Analytic Signal')
    plt.xlim([0, sm_signal.shape[0]])

    
    sm_signal_np[idx,:] = sm_signal

    #axes[idx].set_xticks(ticks = np.arange(0, 80000, 10000), labels = np.arange(0,320,40))
    plt.xlabel('Time [sec]')

    

plt.suptitle('Smoothing Window: 500 samples')
plt.legend(loc='upper right')

plt.show()


In [11]:
x2 = x[5, :] 
sm_stim = window_rms(x2, wintosmooth)
sm_stim1 = (sm_stim)

In [26]:
fig, ax1 = plt.subplots(figsize = (18,6))
#plt.rcParams['font.size'] = 10
ax2 = ax1.twinx()
for idx, dat in enumerate(sm_signal_np):
    ax1.plot(sm_signal_np[idx,:], label = labels[idx], lw = 2)
ax2.plot(sm_stim1, label = 'Stimulation', color = 'grey', ls='--', lw=3, alpha = 0.4)
ax1.legend()
ax1.set_ylabel('Z-scored Smoothed Analytic Signal')
ax2.set_ylabel('Stimulation Amplitude [mA]')
#ax2.set_yticks(np.arange(0.5, 2.5, 0.25))
#ax2.set_yticklabels(np.arange(0.25, 2.25, 0.25))
#ax1.set_xlim(0, sm_signal_np.shape[0])
ax1.set_xticks(np.arange(0, 100000, 20000))
ax1.set_xticklabels(np.arange(0, 400, 80))
ax1.set_xlabel('Time [sec]')
plt.title(str(subID))
plt.show()

In [13]:
sm_analSignal = np.transpose(np.squeeze(np.array([[sm_signal_np[0]], [sm_signal_np[1]],[sm_stim1]])))
sm_analSignal_df = pd.DataFrame(sm_analSignal,
    columns = ['Spontan','StimOn','StimVec'],
    )
print(sm_analSignal_df)

        Spontan    StimOn  StimVec
0      0.914571  0.806174      0.0
1      0.914866  0.804659      0.0
2      0.915780  0.804158      0.0
3      0.916635  0.803850      0.0
4      0.917587  0.803369      0.0
...         ...       ...      ...
94371  0.865956  0.625987      NaN
94372  0.867873  0.629413      NaN
94373  0.870185  0.632879      NaN
94374  0.871988  0.635897      NaN
94375  0.874089  0.639401      NaN

[94376 rows x 3 columns]


In [27]:
fft_fig = os.path.join(project_path, 'figures','FFts','test/')
fft_file = os.path.join(project_path, 'data','FFTs','test/')

plt.savefig(str(fft_fig)+str(fft_name)+'sm_analSignal',dpi = 300)
sm_analSignal_df.to_csv(str(fft_file)+str(fft_name)+'sm_analSignal.csv')

### Averaged Analytic Signal

In [16]:
sm_anal = pd.read_csv(os.path.join(
    project_path,
        'data',
        'FFTs',
        'test',
        'Sub017_sm_analSignal.csv'
))

print(sm_anal)

subID = 'Sub017'
fft_name = str(subID) + '_'
print(fft_name)

       Unnamed: 0   Spontan    StimOn  StimVec
0               0  0.914571  0.806174      0.0
1               1  0.914866  0.804659      0.0
2               2  0.915780  0.804158      0.0
3               3  0.916635  0.803850      0.0
4               4  0.917587  0.803369      0.0
...           ...       ...       ...      ...
94371       94371  0.865956  0.625987      NaN
94372       94372  0.867873  0.629413      NaN
94373       94373  0.870185  0.632879      NaN
94374       94374  0.871988  0.635897      NaN
94375       94375  0.874089  0.639401      NaN

[94376 rows x 4 columns]
Sub017_


In [18]:
261*250

65250

In [41]:
30*250 + 8500

16000

In [37]:
sm_anal1 = sm_anal.iloc[0:7500]
sm_anal2 = sm_anal.iloc[57750:65250]
pad = np.empty((1000,4,))
pad = pd.DataFrame(pad)
sm_toplot = pd.concat([sm_anal1, pad, sm_anal2])
print(sm_toplot)

        0   1   2   3   Spontan    StimOn  StimVec  Unnamed: 0
0     NaN NaN NaN NaN  0.914571  0.806174      0.0         0.0
1     NaN NaN NaN NaN  0.914866  0.804659      0.0         1.0
2     NaN NaN NaN NaN  0.915780  0.804158      0.0         2.0
3     NaN NaN NaN NaN  0.916635  0.803850      0.0         3.0
4     NaN NaN NaN NaN  0.917587  0.803369      0.0         4.0
...    ..  ..  ..  ..       ...       ...      ...         ...
65245 NaN NaN NaN NaN  0.652084  0.756172      2.4     65245.0
65246 NaN NaN NaN NaN  0.653417  0.757206      2.4     65246.0
65247 NaN NaN NaN NaN  0.654988  0.758185      2.4     65247.0
65248 NaN NaN NaN NaN  0.656797  0.759083      2.4     65248.0
65249 NaN NaN NaN NaN  0.658842  0.759884      2.4     65249.0

[16000 rows x 8 columns]


In [50]:
plt.plot(np.arange(0,7500),sm_anal1['Spontan'], color = 'blue')
plt.plot(np.arange(0,7500),sm_anal1['StimOn'], color = 'orange')
plt.plot(np.arange(7500,8500),pad, color = 'white')
plt.plot(np.arange(8500,16000),sm_anal2['Spontan'], color = 'blue', label = 'Peak 77Hz')
plt.plot(np.arange(8500,16000),sm_anal2['StimOn'], color = 'orange', label = 'Peak 63Hz')
plt.ylabel('Z-scored Smooth Analytic Signal')
plt.xlabel('Time [sec]')
plt.legend()

In [25]:
plt.plot(np.arange(0,15000),sm_anal1['Spontan'])
plt.plot(np.arange(0,15000),sm_anal1['StimOn'])
plt.plot(np.arange(0,15000),sm_toplot['StimOn'], label = 'MedOn-StimOn Peak 63Hz')
plt.axvline(x = 7500, color = 'grey', ls='--', lw=3, alpha = 0.4, label = 'Stim On')
plt.xlabel('Time [samples]')
plt.ylabel('Z-scored Smoothed Analytic Signal')
plt.title(str(subID))
plt.legend()
plt.show()


In [17]:
fft_fig = os.path.join(project_path, 'figures','FFts','test/')
fft_file = os.path.join(project_path, 'data','FFTs','test/')

plt.savefig(str(fft_fig)+str(fft_name)+'sm_analSignalCROPPED',dpi = 150)
sm_toplot.to_csv(str(fft_file)+str(fft_name)+'sm_analSignalCROPPED.csv')

NameError: name 'sm_toplot' is not defined

In [51]:
plt.savefig(str(fft_fig)+str(fft_name)+'sm_analSignalCROPPED.pdf')